In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# Assignment 2 - Specification

In [3]:
for task in ["EDA", "ARM", "Regression", "Classification"]:
    content = open(('specification_%s.md' % task).replace(" ","_")).read()
    printmd(content)

### Task - Exploratory Data Analysis

#### Aim

Exploratory Data Analysis is intended to generate insights for subsequent analysis.

As such, its aims are more diffuse. For the Instacart set, much of its purpose is to
create new dataframes by joining the existing tables and looking at typical data, its
cardinality, etc.

#### Detail

Kieran has created a [notebook](Assignment_2_-Setup.ipynb) that shows how to create
your own slices of data from the Instacart data set. It includes settings
like `MAX_PRODUCTS` and `MAX_USERS`, so that is is possible to select a slice that is
specific to you and which is sized appropriately for your laptop.

Regarding EDA, the Python notebooks that Kieran provided for the Week 8 (ARM) Practicals provide an
excellent basis for your own explorations. In particular, [Practical
C](https://kmurphy.bitbucket.io/modules/Data_Mining/topics/08-Association_Analysis/13-Practical_C_-_ARM_Instacart_Dataset/files/Practical_C_-_ARM_Instacart_Dataset.ipynb)
and parts of Practicals A and B should be used as a starting point for your own
investigations.

For the classification task, Kieran has also studied reorder rate as a function
of "order added to cart", see ![Reorder-rate vs Order-added-to-cart](add_order_v_reorder_rate.png).

We recommend that you recreate his analysis and use linear regression to derive the
relationship between reorder rate and the order in which products are added to each cart,
as this represents a feature that might prove helpful when predicting the product mix
of the next order (which is the subject of the Classification task).



### Task - Association Rule Mining 

#### Aim

Perform an association rule mining (ARM) analysis (of your slice) of the Instacart dataset.

#### Detail

Perform ARM using __product__, __aisle__, and __department__ as the item.

To perform an ARM with item=product we need to reduce the number of products (from the initial 50K products). A crude approach is to simply drop all product which appear less than some minimum frequency threshold (say __PRODUCT_KEEP_MIN_FREQ=500__).

The table __orders_products__ needs to be one-hot encoded. Simplest way to do this is to first convert the order-product pairs to a transaction list and then (as in Practical B) to a one-hot encoding.  

~~~~.python
# convert table of (order,product) pairs to list of transactions
transactions = orders_products.groupby('order_id').apply(lambda order: order['product_id'].tolist())
~~~~

For each ARM with item (product, aisle, department) you should at least do the following subtasks:

 * Visualise the first 100 transactions vs items and comment (see Practical B).
 * Visualise the distribution of transaction size and comment (see Practical B).
 * Generate a few hundred frequent itemsets. You need to experiment (as discussed in class) to estimate a suitable value for the support threshold.
 * Generate a hundred+ rules and see if your can identify any interesting ones.

#### Disclaimers/Comments

 * When using __aisle__ and __department__ for item in the ARM model, we really should be using more advanced models than just (0=absent, 1=present).  Feel free to explore the more advanced models, but we would be happy with just the one-hot analysis.
 
 * When you perform ARM with __item=product__ we will find the your rule list will swamped by the organic fanboy/girl customers (my term not Bernard's) and I realy hope those customers buying the $\{lemon, lime\}$ itemset were doing it for non-healthy reasons like drinking gin! 
 
 * A more sophisticated analysis with __item=product__ would be to roll up (merge) essentially similar products. So 
   * All the organic fruit (${Organic\ Strawberries, Organic\ Raspberries, Bag\ of\ Organic\ Bananas, \ldots looong\ list\ldots }$) would just go to "organic fruit"
   * Merge products which differ only by brand name. 
   
#### Grading Outline

 * 3 x 25% for each of the separate basic ARM with item = (product, aisle, department) where the 25% consists of  
   * 10% Visualisation 
   * 5% Rule generation 
   * 10% Comments/analysis and identification of interesting rules.
 * 25% for performing some roll-up of products and a more sophisticated analysis.





 






### Task - Linear regression 

#### Aim

Perform a linear regression analysis (of your slice) of the Instacart dataset,
predicting the number of days between orders.

#### Detail

The problem to solve takes the form of _Linear Regression with categorical
independent variables_.

Consider the sequence of orders placed by a customer. The hypothesis is that
the typical product mix for that customer can be used to predict when the next
order will be placed.

For training data, take an 80% subset of "relevant" __order__s, across all
"relevant" __user__s. From the first to the second last order, determine the
__product__ mix for that __user__, as well as the __number of days__ until the
next __order__ for that __user__.

The training data is the remaining 20% subset of "relevant" __order__s, across
all "relevant" __user__s. 

In practice, _relevant_ implies an unbiased subset chosen to make the problem
tractable with the computing resources available to each student.

For _Independent variables_, the __product__ (mix) for __order__ `i-1` can be
used, and the _Dependent variable_ is the __number of days__ between __order__
`i-1` and __order__ `i` in that sequence for a given __user__.

Therefore number of observations is the number of __user__s times the number of
__order__s for each __user__.

Since there are 49688 distinct products, the dimensionality of the problem is
very high, so students need to take account of this using:

1. roll up to __aisle__ (134 unique) or __department__ (21 unique) and build
   model accordingly 
2. use PCA to reduce the dimensions (of __product__s, __aisle__s and/or
   departments), and solve using the reduced-dimension model
3. use regularisation and fit a constrained model.

#### Disclaimers/Comments

* It is possible to obtain either per-user predictions (by solving a separate
  regression problem for each user) or a user-agnostic prediction (by merging
  the data for many users). It would be interesting to try both and compare,
  explaining what you find. 
 
* Much of this task relates to dimensionality reduction. In traditional BI,
  this would be done using roll-ups (e.g., from __product__ to __aisle__), but
  linear regression offers other options. The BI approach is easier to
  interpret, because of the "meaningful" labels, but does it work as well? Give
  reasons fior your answer.

* Validating the results requires analysis of the outputs (and not just the
predictions) of the linear regression model, e.g., R-squared and any other
metrics you believe are helpful here.
   
#### Grading Outline

 * 3 x 25% for each of the 3 regression model x dimensionality reduction options above, where the 25% consists of  
   * 5% Visualisation 
   * 10% formulation and solving 
   * 10% Validation/Comments/analysis.
 * 25% for comparing per-user and multi-user models, and different methods for dimensionality reduction.


### Task - Classification 

#### Aim

Multi-label classification with numerical variables (of your slice) of the
Instacart dataset, predicting the product mix in the next order for a user.

#### Detail

The problem to solve takes the form of _Classification with numerical
independent variables_.

Given the history of __order__s per user, Instacart would like to predict the
mix of __product__s that would be ordered next. From a business perspective,
there is little to be gained from promoting a product that would be purchased
in any case, but much to be gained from promoting a related product. Therefore,
knowing the baseline (i.e., without marketing intervention) could improve the
efficiency of product marketing efforts.

As with the regression task, we recommend that you choose a _relevant_
subset of users for analysis, where _relevant_ implies an unbiased subset
chosen to make the problem tractable with the computing resources available to
each student.

There are two subtasks that need to be considered:
1. taking historical order data from (a subset of) users and using this data
to predict the next order for a typical user, given that user's order history
2. taking historical order data from a _single_ user and using this data to
predict the next order for that user.

The basic hypothesis in the first of these is that there is an underlying
consistency across users in terms of their order behaviour, so the more data
that is available, the better the prediction of a typical user's next order.

By contrast, the hypothesis in the second of these is that the order behaviour
of different differs from each other, so it is better to tailor next order
predictions to individual users.

We recommend the following three features for each __product__:

1. relative quantity across all orders for that user (computed as the number of
times it is ordered divided by the number of all products ordered)
2. relative frequency of reorder (so if ordered twice in 4 orders, the reorder
rate is (2-1)/(4-1) = 1/3). If not reordered, it takes the value 0.
3. (some function of) "add to cart order", across all orders for a given user. For example, if it
was ordered 3 times, and was the first, third and fifth item added to these orders,
the corresponding feature would take the _mean_ value. Optionally, students might wish to investigate
different ways of calculating this mean: the traditional
[arithmetic mean](https://en.wikipedia.org/wiki/Arithmetic_mean) (3 in this case) or the
[harmonic mean](https://en.wikipedia.org/wiki/Harmonic_mean) (approximately 1.96 in this case)
that gives more weight to smaller values.

The 3 features above are chosen in an attempt to capture the following predictive attributes:
* how often a product is ordered, compared to all other products for that user
* how frequently a product is reordered by a given user
* using "add to cart order" as a proxy to represent the importance of the product to the user

Regarding feature 3, Kieran analysed the reorder rate as a function of "add to cart order"
and found that it declined steadily as the "add to cart order" increased. This trend
stopped abruptly at "add to cart order" 35 (approximately) when the relationship broke down,
by exhibiting large oscillations.

Therefore, we recommend that you, as part of the Exploratory Data Analysis phase, plot
reorder rate (across all _relevant_ products and users) against "add to cart order", and
use this to determine when the simple relationship breaks down, say at n=35.

You can then use linear regression to fit a low-order polynomial (linear or quadratic, say)
to reorder rate as a function of "add to cart order" for the restricted range while the
relationship shows a stable trend. Using the linear regression you learned, you should then
be able to extrapolate a reorder rate for higher "add to cart order" values.

Indeed, it should be possible to use the results of this linear regression (and
its extrapolation to higher "add to cart order" values) to
lookup a generic reorder rate for a given (mean) "add to cart order" for a given
__product__ for a given __user__. This generic reorder rate can then be used in
place of feature 3 above, as it is derived from the same data and represents
much the same behaviour.

The _dependent_ variable is the product mix in the next order for that user in the
training set, expressed as a set of binary variables (0,1-valued: product is included or not).

The number of __product__s could cause difficulties, because the number of
features (hence columns in the observation matrix) is 3 times the number of __product__s
considered in the model, and the dependent variable is vector-valued, comprising
indicator values for each of those products.

Therefore, you are recommended to limit your analysis as follows:
1. with multiple __user__s, take the most popular 200 products, say
2. with a single __user__, limit to the __product__s purchased by that user in their
orders to date.

Regarding training, test data and validation, we recommend the following:

1. given order data from multiple users, the training data comprises all orders
for an 80% subset of the _relevant_ users. The learned classifier can then
be applied to order data from the remaining 20% subset of users. For
validation, you need to measure how well the classifer predicts the last order
for those "test" users.
2. given order data from a single user, the training data for that user excludes the
last order for that user. For testing, the learned classifier is
applied to all the data including the last order and the __product__ mix
of the last order is predicted. For validation, the predicted last order
can be compared with the actual last order for that user.

#### Disclaimers/Comments

* Feature engineering is a very important part of this task, because Instacart does
not provide labels as part of the data. Therefore, they need to be derived.
 
* The classifiers covered in this module generally do not support as many options
for dimensionality reduction as linear regression does, so depending on the scope
of the learned classifier, we recommend either a user-specific product subset, or
a "Top N" approach (of products purchased by the _relevant_ users) in this task.
Needless to say, it is essential to ensure that the same subset of __product__s
is used throughout the task.

* One of the objectives in this task is to determine whether per-user or
across-user models make better predictions. Therefore, you will need to
compare the two model scopes and suggest which is more applicable to the
Instacart data.

* Multilabel classification, as used here, is most commonly used in document
classification. For example, we might wish to label a book with multiple non-exclusive
attributes, such as its theme, its genre, its intended audience, and whether it
belongs to a series or not. When used for market basket analysis, the main difficulty
with multilabel classification is that the number of such labels (identifying products
uniquely) can be enormous.

#### Grading Outline

 * 25% deriving features 1 and 2 (will require data manipulation)
 * 25% deriving feature 3 (will require solving a linear regression sub-problem)
 * 20% choosing and using a classifier
 * 30% validating the results, to include analysis of confusion matrix, precision, recall and between-model metrics such as F1-score etc.



# Submitting your attempt

We recommend that you submit 4 jupyter notebooks, each addressing the main tasks above.

Your notebook should be fully annotated: you should explain any code and interpret any results. Even if a particular line of analysis does not seem to work for you, please explain what you did and why you believe it is not working. Marks will be given for any reasonable attempt - we cannot assign marks if you do not provide your "workings" at least!

The tasks have different weightings, reflecting their different levels of difficulty and the time needed to complete them:

   * EDA.     (20%)
   * Model to predict days to next order (25%)
   * ARM.    (15%)
   * Classification to predict next order product mix (40%)

Note that some EDA will be needed in support of the other 3 tasks, so you might choose to move your EDA efforts into the relevant task notebooks, but you should still indicate (via section headings and other annotations) what parts of those notebooks relate to EDA.
